In [ ]:
import io
import re
import smtplib
import openpyxl
import getpass
import requests
import locale
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def extract_data():
    print("Extraindo dados do Google Drive...")
    try:
        url = 'https://drive.google.com/drive/folders/14oLE59U1RqyRqlBbKpsyymW-mitvbtoh'
        response = requests.get(url)
        file_id = re.findall(r'data-id="([a-zA-Z0-9_-]{28,})"', response.text)[0]
        url_file = f'https://drive.google.com/uc?id={file_id}&export=download'
        content = io.BytesIO(requests.get(url_file).content)
        df = pd.read_excel(content)
        total_qty = df['Quantidade'].sum()
        total_val = df['Valor Final'].sum()
        locale.setlocale(locale.LC_ALL, '')
        total_val_formatted = locale.format_string("%.2f", total_val, grouping=True)
        print("Dados extraídos com sucesso!\n")
        return total_qty, total_val_formatted
    except Exception as e:
        print("Erro ao extrair dados:", e)
        return None

def send_email(email, password, recipients, subject, total_qty, total_val):
    print("Enviando e-mail...")
    body = f'Prezados, Bom dia.\n\nO faturamento de ontem foi de: R$ {total_val}\nA quantidade de produtos ' \
           f'vendidos foi de: {total_qty}\n\nAbs\nAllan Santos'
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = ", ".join(recipients)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    try:
        server = smtplib.SMTP('smtp.office365.com', 587)
        server.starttls()
        server.login(email, password)
        password = ''
        text = msg.as_string()
        server.sendmail(email, recipients, text)
        server.quit()
        print("E-mail enviado com sucesso!")
    except Exception as e:
        print("Erro ao enviar e-mail:", e)
        exit(1)

if __name__ == '__main__':
    email = input('Endereço de e-mail do remetente: ')
    password = getpass.getpass('Senha do remetente: ')
    recipients = input('Endereços de e-mail dos destinatários separados por vírgula: ').split(',')
    subject = input('Assunto do e-mail: ') if input(
        'Deseja personalizar o assunto - (s/n). ').lower() == 's' else 'Faturamento '

    data = extract_data()
    if data:
        total_qty, total_val = data
        send_email(email, password, recipients, subject, total_qty, total_val)
    else:
        print("Não foi possível extrair os dados do Google Drive.")

Endereço de e-mail do remetente: allan.santos59@fatec.sp.gov.br
Senha do remetente: ··········
Endereços de e-mail dos destinatários separados por vírgula: allannascimentodossantos@gmail.com
Deseja personalizar o assunto - (s/n). n
Extraindo dados do Google Drive...
Dados extraídos com sucesso!

Enviando e-mail...
E-mail enviado com sucesso!
